In [ ]:
#TabulaRazr - specific to calculate  - TABLE Parser
#Infers a table with arbitrary number of columns from reoccuring patterns in text lines
#(c) Alexander Hirner 2016, no redistribution without permission
#Contributions: ____ (refactoring), UI styling (), ....


#Main assumptions Table identificatin:
#1) each row is either in one line or not a row at all
#2) each column features at least one number (=dollar amount)
#2a) each column features at least one date-like string [for time-series only]
#3) a table exists if rows are in narrow consecutive order and share similarities --> scoring algo [DONE] 
#4) each column is separated by more than x consecutive whitespace indicators (e.g. '  ' or '..')

#Feature List Todo:
#1) Acknowledge footnotes / make lower meta-data available
#2) make delimiter length smartly dependent on number of columns (possible iterative approach)
#3) improve captioning: expand non canonical values in tables [DONE] .. but not to the extent how types match up  --> use this to further
## delineate between caption and headers
#4) UI: parameterize extraction on the show page on the fly
#5) deeper type inference on token level: type complex [DONE], subtype header (centered, capitalized), 
## subtype page nr., type free flow [DONE, need paragraph]
#6) Respect negative values with potential '-' for numerical values
#7) 
#8) classify tables with keywords (Muni Bonds) and unsupervised clustering (Hackathon)
#9) Restructure folder and URI around MD5 hash (http://stackoverflow.com/questions/24570066/calculate-md5-from-werkzeug-datastructures-filestorage-without-saving-the-object)

In [1]:
from backend import *
import os

In [2]:
from fuzzywuzzy import fuzz
#Filter and Rank Tables upon string data:

#print (fuzz.partial_ratio("this is a test!", "this text is"))


#print (fuzzy_str_match("aaaaaaaaaa text this", "this test is"))

print (fuzzy_str_match("USES OF FUNDS", "TABLE OF CONTENTS"))
print (fuzzy_str_match("USES OF FUNDS", "Note 5 - Assets Limited as to Use"))
print (fuzzy_str_match("USES OF FUNDS", "FINANCIAL STATEMENT AMOUNTS - Continued"))


('penalty', 1.0)
0.27
('penalty', 1.0)
0.27
('penalty', 1.0)
None


In [8]:
print (fuzzy_str_match("Maturity", "ii"))
#print (fuzz.partial_ratio("Maturity", "i"))
print (fuzzy_str_match("Maturity", "Matarit"))
print (fuzzy_str_match("Maturity", "i"))
print (fuzzy_str_match("Maturity", "     i   "))



print ("Maturity".lower() in "i".lower())

('penalty', 0.125)
None
('penalty', 0.75)
0.3225
('penalty', 0.0)
None
('penalty', 1.0)
None
False


In [ ]:
def run_from_ipython():
    try:
        __IPYTHON__
        return True
    except NameError:
        return False

if run_from_ipython():
    app.run(host='0.0.0.0', port = 8080)
else:
    PORT = int(os.getenv('PORT', 8080))
    app.run(debug=True, host='0.0.0.0', port = PORT)